In [9]:
import numpy as np
from random import randint, random
from copy import deepcopy
from tqdm.auto import tqdm


/home/michele/anaconda3/envs/ml/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
#Main optimization function: Simulated Annealing
def simulated_annealing_knapsack(VALUES, WEIGHTS, CONSTRAINTS, *,
                                 max_steps=2000,
                                 init_temp=100.0,
                                 cooling_rate=0.995,
                                 tweak_strength=0.3,
                                 penalty_factor=10.0,
                                 verbose=True):
   

    NUM_ITEMS = len(VALUES)
    NUM_KNAPSACKS, NUM_DIMENSIONS = CONSTRAINTS.shape

   #Generate a random solution
    def random_solution():
        #Each item assigned randomly to one knapsack or -1 (not taken)
        return [randint(-1, NUM_KNAPSACKS - 1) for _ in range(NUM_ITEMS)]

    #Evaluate a solution
    def cost(solution):
        #Negative total value + penalties for capacity violations
        penalty = 0
        total_value = 0
        for k in range(NUM_KNAPSACKS):
            used = np.zeros(NUM_DIMENSIONS)
            for i, s in enumerate(solution):
                if s == k:
                    used += WEIGHTS[i]
                    total_value += VALUES[i]
                    #Add penalties for violated constraints
            for d in range(NUM_DIMENSIONS):
                if used[d] > CONSTRAINTS[k, d]:
                    penalty += penalty_factor * (used[d] - CONSTRAINTS[k, d]) / (CONSTRAINTS[k, d] + 1e-9)
        return -total_value + penalty
    
    #Create a modified solution (small perturbation)
    def tweak(solution, strength=tweak_strength):
        new_solution = deepcopy(solution)
        n_changes = max(1, int(len(solution) * strength))
        for _ in range(n_changes):
            i = randint(0, NUM_ITEMS - 1)
            new_solution[i] = randint(-1, NUM_KNAPSACKS - 1)
        return new_solution

   #Simulated Annealing loop
    current_solution = random_solution()
    current_cost = cost(current_solution)
    best_solution, best_cost = deepcopy(current_solution), current_cost
    temp = init_temp

    for step in tqdm(range(max_steps), disable=not verbose):
        #Gradually decrease tweak intensity
        new_solution = tweak(current_solution, strength=max(0.05, tweak_strength * (1 - step / max_steps)))
        new_cost = cost(new_solution)
        diff = new_cost - current_cost
 
        #Accept better solutions or occasionally worse ones (exploration)
        if new_cost < current_cost or random() < np.exp(-diff / temp):
            current_solution, current_cost = new_solution, new_cost
            if current_cost < best_cost:
                best_solution, best_cost = deepcopy(current_solution), current_cost

        #Temperature decay
        temp *= cooling_rate

    return best_solution, best_cost


In [11]:
#Evaluate and print a final solution
def evaluate(solution, VALUES, WEIGHTS, CONSTRAINTS):
    NUM_KNAPSACKS, NUM_DIMENSIONS = CONSTRAINTS.shape
    total_value = 0

    for k in range(NUM_KNAPSACKS):
        used = np.zeros(NUM_DIMENSIONS)
        value = 0
        items = []
        for i, s in enumerate(solution):
            if s == k:
                used += WEIGHTS[i]
                value += VALUES[i]
                items.append(i)
        total_value += value
        print(f"Knapsack {k}: items={items[:10]}{'...' if len(items) > 10 else ''}, "
              f"used={used.round(1)}, capacity={CONSTRAINTS[k]}, value={value}")
    print("Items not taken:", [i for i, s in enumerate(solution) if s == -1][:20])
    print(f"Total value: {total_value}")


In [12]:
#Problem 0
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 10
NUM_DIMENSIONS = 2

VALUES = rng.integers(10, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(5, 50, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS))

solution, cost_val = simulated_annealing_knapsack(
    VALUES, WEIGHTS, CONSTRAINTS,
    max_steps=1000,
    init_temp=100,
    cooling_rate=0.995,
    verbose=True
)

print("\n--Problem 0 Results--")
print("Best cost:", cost_val)
evaluate(solution, VALUES, WEIGHTS, CONSTRAINTS)


100%|██████████| 1000/1000 [00:00<00:00, 14807.26it/s]


--Problem 0 Results--
Best cost: -455.7619047623879
Knapsack 0: items=[1, 2, 3, 5, 7, 8, 9], used=[222. 209.], capacity=[150  75], value=401
Knapsack 1: items=[0, 6], used=[41. 94.], capacity=[ 30 184], value=35
Knapsack 2: items=[4], used=[42. 25.], capacity=[295  21], value=48
Items not taken: []
Total value: 484


In [16]:
print(CONSTRAINTS)

[[79834 52005 17527 ... 78500 56643 69915]
 [95067 28572 45458 ... 50816 83737 88901]
 [46612 36250 19604 ... 98589 28135 27193]
 ...
 [69672 90152 96973 ... 43906 41364 99258]
 [60364 83491 71527 ... 33636 13114 12777]
 [49805 58803 31629 ... 83824 73369 10897]]


**TEST PROBLEMS**

In [13]:
#Problem 1
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS))

solution, cost_val = simulated_annealing_knapsack(
    VALUES, WEIGHTS, CONSTRAINTS,
    max_steps=2000,
    verbose=True
)

print("\n--Problem 1 Results--")
print("Best cost:", cost_val)
evaluate(solution, VALUES, WEIGHTS, CONSTRAINTS)


100%|██████████| 2000/2000 [00:00<00:00, 16947.44it/s]


--Problem 1 Results--
Best cost: -1065
Knapsack 0: items=[2, 4, 5, 6, 7, 8, 12, 15, 18, 19], used=[568. 472.], capacity=[614 496], value=569
Knapsack 1: items=[1, 3, 10, 11, 13, 16], used=[224. 419.], capacity=[244 644], value=396
Knapsack 2: items=[0, 9, 14, 17], used=[193. 181.], capacity=[273 216], value=100
Items not taken: []
Total value: 1065


In [14]:
#Problem 2
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS))

solution, cost_val = simulated_annealing_knapsack(
    VALUES, WEIGHTS, CONSTRAINTS,
    max_steps=3000,
    cooling_rate=0.998
)
print("\n--Problem 2 Results--")
print("Best cost:", cost_val)
evaluate(solution, VALUES, WEIGHTS, CONSTRAINTS)


  0%|          | 0/3000 [00:00<?, ?it/s]

100%|██████████| 3000/3000 [00:00<00:00, 3377.31it/s]


--Problem 2 Results--
Best cost: -52542.27182304506
Knapsack 0: items=[12, 19, 29, 32, 34, 35, 43, 74], used=[5426. 3639. 4485. 2545. 4088. 3273. 3637. 3771. 3855. 2239.], capacity=[5837 5754 9351 3205 4450 3447 3952 9256 9525 2357], value=3710
Knapsack 1: items=[3, 7, 13, 18, 46, 53, 73, 84, 88], used=[4205. 3701. 4390. 4353. 5064. 4888. 5003. 2387. 4738. 5531.], capacity=[7056 3862 3540 4336 3555 5921 9964 6691 7785 5946], value=5171
Knapsack 2: items=[24, 48, 49, 54, 77, 85, 92], used=[2801. 3518. 3415. 3417. 4702. 3478. 3735. 4008. 4688. 3872.], capacity=[2111 2672 4150 3949 8724 8748 8832 7100 6449 7193], value=4500
Knapsack 3: items=[1, 11, 14, 15, 17, 27, 33, 37, 41, 42]..., used=[4836. 7818. 6390. 6166. 9338. 4418. 8224. 5289. 7070. 5513.], capacity=[5254 7361 8355 8103 6910 2464 7799 4932 7949 6316], value=8116
Knapsack 4: items=[8, 21, 26, 36, 45, 55, 63, 65, 69, 81]..., used=[9419. 6035. 6993. 8891. 4975. 7409. 8372. 7141. 5477. 8115.], capacity=[9049 4707 9451 8755 4648 58

In [15]:
#Problem 3
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS))

solution, cost_val = simulated_annealing_knapsack(
    VALUES, WEIGHTS, CONSTRAINTS,
    max_steps=10000,
    init_temp=500,
    cooling_rate=0.999,
    tweak_strength=0.06,
    verbose=True  
)
print("\n--Problem 3 Results--")
print("Best cost:", cost_val)
evaluate(solution, VALUES, WEIGHTS, CONSTRAINTS)


100%|██████████| 10000/10000 [04:59<00:00, 33.40it/s]



--Problem 3 Results--
Best cost: -2482359.1487197657
Knapsack 0: items=[103, 105, 177, 508, 617, 626, 781, 833, 854, 881]..., used=[27410. 28685. 23576. 25557. 24570. 25693. 23532. 27291. 25370. 24735.
 23764. 27661. 24955. 27554. 24095. 25607. 26389. 26720. 26236. 25441.
 25063. 26421. 22705. 31086. 23836. 25072. 21787. 26269. 29185. 28942.
 27216. 29302. 27828. 32153. 26748. 25482. 25396. 26047. 27705. 26213.
 26853. 27945. 24721. 25806. 26656. 28664. 27550. 22676. 27566. 24625.
 25109. 25307. 24872. 27269. 24508. 26768. 24573. 23464. 31844. 25146.
 23669. 30509. 24365. 25547. 24639. 25534. 27567. 28952. 27375. 22544.
 29760. 28327. 27858. 21856. 26105. 24611. 26415. 24269. 27498. 23586.
 29147. 29388. 26498. 25122. 26514. 24106. 25823. 23069. 26681. 23460.
 24085. 30113. 24903. 27080. 29900. 25865. 27403. 20411. 22276. 24999.], capacity=[79834 52005 17527 30007 59295 98921 12881 36174 63958 90566 25740 12251
 27746 78136 90704 39249 23072 69010 17326 29277 42886 48016 74931 93479
 